In [95]:
autumn['DATE'] = pd.to_datetime(autumn['DATE'])

In [2]:
autumn = pd.read_csv('autumn2.csv')

In [4]:
autumn['RANGE'] = autumn['TMAX'] - autumn['TMIN']

In [5]:
#maybe this is the database?
conn = sqlite3.connect('weather_data.db')
autumn.to_sql('lou_weather',conn)

/Users/ryanpierson/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2127: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


ValueError: Table 'lou_weather' already exists.

In [8]:
autumn.loc[autumn['RANGE'].idxmax()]
autumn.loc[autumn['RANGE'].idxmin()]

DATE     2010-10-09
TMAX             87
TMIN             47
RANGE            40
YEAR           2010
MONTH            10
WEEK             40
DAY               9
Name: 38, dtype: object

In [40]:
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, show
from bokeh.models import HoverTool
from bokeh.plotting import figure
import pandas as pd
import sqlite3

output_notebook()
#Gets the distinct years from the database


def main(year):
    conn = sqlite3.connect('weather.db')
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS maxminall''')
    c.execute('''CREATE TABLE maxminall AS 
                Select concat, 
                    MONTH, 
                    DAY,
                    YEAR,
                    MAX(TMAX) over_max, 
                    MIN(TMIN) over_min  
                from (select AVG(TMAX) tmax, 
                            AVG(TMIN) tmin,
                            MONTH || DAY concat,
                            MONTH,
                            DAY,
                            YEAR
                        from weather
                        where YEAR > '''+ str(year) +''' 
                        GROUP BY 3, 4, 5, 6)
                        GROUP BY 1, 2, 3, 4''')
    
    #print(pd.read_sql('select * from maxminall', conn))
                
    #Retrieves the range for the plots
    rng = pd.read_sql('select MAX(TMAX) max, MIN(TMIN) min from weather', conn)
    mx, mn =(rng.loc[0, 'max'], rng.loc[0,'min'])
    #print(pd.read_sql('select * from maxminall', conn))
    #return pd.read_sql(quer(year), conn)
    display_all_plots(sql_dict(year))
    

# # Query to combine the overall metrics with the individual table
# def quer(y):
#     return'''
#     select a.DATE, 
#     (AVG(a.TMAX)+ AVG(a.TMIN))/2 avg_temp,  
#     a.YEAR, 
#     a.MONTH, 
#     a.DAY, 
#     (b.over_max + b.over_min)/2 comp  
#     from weather a join maxminall b
#     on a.MONTH || a.DAY = b.concat WHERE a.MONTH > 8 AND a.YEAR = ''' + str(y) + " GROUP BY 1, 3, 4, 5, 6"


# Query to combine the overall metrics with the individual table
def quer(y):
    return'''
    select a.DATE, 
    AVG(a.TMAX) TMAX,
    AVG(a.TMIN) TMIN,  
    a.YEAR, 
    a.MONTH, 
    a.DAY, 
    MAX(b.over_max) overmax, 
    MIN(b.over_min) overmin,
    b.concat
    from weather a 
    join 
    maxminall b
    on a.MONTH || a.DAY = b.concat WHERE a.MONTH > 8 AND a.YEAR = ''' + str(y) + " GROUP BY 1, 4, 5, 6, 9"





#Creates the max/min/all temp table


# New Version
def sql_dict(year):
    #query = "Select Distinct YEAR from weather WHERE YEAR > (strftime('%Y','now') - 10) ORDER BY YEAR"
    query = ("Select Distinct YEAR from weather WHERE YEAR > "+str(year) +" ORDER BY YEAR")
    yr = pd.read_sql(query, conn)
    #print("part two:", yr)
    sql_results = {}
    for i in range(len(yr) - 1): 
        y = yr.loc[i,'YEAR']
        #print(y)
        sql_results[str(y)] = {}
        sql_results[str(y)]['DATA'] = pd.read_sql(quer(y), conn)
        sql_results[str(y)]['DATA']['DATE'] = pd.to_datetime(sql_results[str(y)]['DATA']['DATE'])
    #print(pd.read_sql('select * from maxminall', conn))
    return sql_results

#makes the plot
#works
def plot_maker(data, div): 
    hover = HoverTool()
    hover.tooltips = [
        ("index", "$index"),
        ("(x,y)", "($x, $y)"),
        ("radius", "@radius"),
        ("fill color", "$color[hex, swatch]:fill_color"),
        ("foo", "@foo"),
        ("bar", "@bar"),
    ]
    rng = pd.read_sql('select MAX(TMAX) max, MIN(TMIN) min from weather', conn)
    mx, mn =(rng.loc[0, 'max'], rng.loc[0,'min'])
    p = figure(x_axis_type='datetime',y_range=(mn,mx) ,width=500, plot_height=250, title=div)
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_color='red'
    p.ygrid.grid_line_alpha=0.5
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = "Temperature"
    p.line(data[div]['DATA'].DATE, data[div]['DATA'].TMAX, line_width=2, color='red')
    p.line(data[div]['DATA'].DATE, data[div]['DATA'].TMIN, line_width=2, color='blue')
    p.line(data[div]['DATA'].DATE, data[div]['DATA'].overmax, line_width=2, color='green', alpha=0.5)
    p.line(data[div]['DATA'].DATE, data[div]['DATA'].overmin, line_width=2, color='purple', alpha=0.5)
    p.tools.append(hover)
    return p

# Grouping 
def data_grid(lst, col=2):
    output = []
    for i in range(0, len(lst), col):
        if (i + (col - 1)) >= len(lst):
            output.append(lst[i:])
        else:
            output.append(lst[i:(i+(col))])
    return output


def display_all_plots(data):
    viz = []
    for i in data.keys():
        #print(i)
        viz.append(plot_maker(data, i))
    show(gridplot(data_grid(viz)))
    #show(data_grid(viz))

Loading BokehJS ...

In [41]:
main(2016)

In [8]:
pd.read_sql('select * from maxminall', conn)

NameError: name 'conn' is not defined

In [136]:
conn = sqlite3.connect('weather.db')
c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS maxminall''')
c.execute('''CREATE TABLE maxminall AS 
            Select concat, 
                MONTH, 
                DAY,
                YEAR,
                TMAX over_max, 
                TMIN over_min
                from (select AVG(TMAX) tmax, 
                        AVG(TMIN) tmin,
                        MONTH || DAY concat,
                        MONTH,
                        DAY,
                        YEAR
                    from weather
                    where YEAR > '''+ str(year) +''' 
                    GROUP BY 3, 4, 5, 6)''')
    

In [100]:
sql_dict(2016)

part two:    YEAR
0  2017
1  2018


{'2017': {'DATA':           DATE   TMAX   TMIN  YEAR  MONTH  DAY    overmax    overmin
  0   2017-09-01  76.50  56.25  2017      9    1  82.666667  65.333333
  1   2017-09-02  64.50  55.00  2017      9    2  79.083333  62.916667
  2   2017-09-03  78.75  54.50  2017      9    3  84.666667  62.833333
  3   2017-09-04  85.75  64.75  2017      9    4  88.583333  66.083333
  4   2017-09-05  79.25  59.00  2017      9    5  87.166667  66.416667
  5   2017-09-06  73.00  51.25  2017      9    6  83.500000  63.833333
  6   2017-09-07  70.75  49.50  2017      9    7  83.333333  64.250000
  7   2017-09-08  76.75  51.25  2017      9    8  82.750000  63.833333
  8   2017-09-09  77.00  54.00  2017      9    9  79.083333  63.750000
  9   2017-09-10  76.25  53.00  2017      9   10  78.750000  60.000000
  10  2017-09-11  71.00  53.00  2017      9   11  74.583333  56.500000
  11  2017-09-12  74.25  57.50  2017      9   12  79.454545  58.727273
  12  2017-09-13  66.75  60.75  2017      9   13  80.166667  

In [91]:
pd.read_sql('select * from maxminall', conn)

,concat,MONTH,DAY,over_max,over_min
0,101,10,1,81.000,54.500000
1,1010,10,10,79.750,68.000000
2,1011,10,11,74.625,55.250000
3,1012,10,12,68.250,58.500000
4,1013,10,13,72.750,53.750000
5,1014,10,14,81.750,54.250000
6,1015,10,15,80.000,54.000000
7,1016,10,16,68.000,48.500000
8,1017,10,17,66.000,42.750000
9,1018,10,18,72.750,43.750000


In [ ]:
# # plot maker works -- displays the graph
# show(plot_maker(data=sql_results, div='2010'))

### -----
from bokeh.layouts import gridplot

viz = []
for i in sql_results.keys():
    viz.append(plot_maker(sql_results, i))
show(gridplot(data_grid(viz)))

In [10]:
df = sql_dict(2010)

DatabaseError: Execution failed on sql 'Select Distinct YEAR from weather WHERE YEAR >2010ORDER BY YEAR': unrecognized token: "2010ORDER"

In [111]:
def rew(year):
    test = '''
    select 
        MAX(a.over_max),
        MAX(a.over_min),
        AVG(b.TMAX),
        AVG(b.TMIN),
        b.Month,
        b.day,
        a.concat
        FROM
    (select 
                AVG(TMAX) over_max, 
                AVG(TMIN) over_min,
                MONTH || DAY concat,
                MONTH,
                DAY
        from weather
        where YEAR > '''+ str(year) +''' 
        and MONTH > 8
        GROUP BY 3, 4, 5) a join weather b
        on a.concat = b.Month || b.day
        where b.YEAR > '''+ str(year) +'''
        GROUP BY 5,6, 7'''
    return(pd.read_sql(test, conn))
rew(2016)

,MAX(a.over_max),MAX(a.over_min),AVG(b.TMAX),AVG(b.TMIN),MONTH,DAY,concat
0,58.500,36.750000,62.875000,42.875000,1,11,111
1,69.250,53.750000,64.250000,32.625000,1,12,112
2,69.250,52.250000,40.000000,24.000000,1,13,113
3,69.000,51.000000,32.250000,22.125000,1,14,114
4,76.000,59.000000,35.750000,25.375000,1,15,115
5,73.250,49.750000,36.875000,24.000000,1,16,116
6,52.000,46.250000,40.750000,24.875000,1,17,117
7,52.750,39.000000,39.375000,24.125000,1,18,118
8,58.250,33.750000,46.250000,28.750000,1,19,119
9,56.250,31.500000,61.500000,46.375000,1,21,121


In [55]:
#c.execute('DROP TABLE if exists maxminall')
# year = 2015
# c.execute('''CREATE TEMPORARY TABLE maxminall AS Select MONTH || DAY concat, MONTH, DAY,MAX(TMAX) over_max, MIN(TMIN) over_min 
#                 from weather where YEAR > '''+ str(year) +''' GROUP BY 1, 2, 3 ORDER BY 2, 3''')
    
pd.read_sql('select * from maxminall', conn)

DatabaseError: Execution failed on sql 'select * from maxminall': no such table: maxminall

In [5]:
conn = sqlite3.connect('weather.db')
c = conn.cursor()
c.execute('''CREATE TEMPORARY TABLE maxminall AS Select MONTH || DAY concat, MONTH, DAY,MAX(TMAX) over_max, MIN(TMIN) over_min 
            from weather GROUP BY 1, 2, 3 ORDER BY 2, 3''')
#Retrieves the range for the plots
rng = pd.read_sql('select MAX(TMAX) max, MIN(TMIN) min from weather', conn)
mx, mn =(rng.loc[0, 'max'], rng.loc[0,'min'])



In [37]:
pd.read_sql("select MONTH, DAY from weather where YEAR > 2010 limit 5", conn)

,MONTH,DAY
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5


In [76]:
def query(sql):
    return pd.read_sql(sql, conn, index_col='index')

In [236]:
# Query to combine the overall metrics with the individual table
def quer(y):
    return'''select a.DATE, 
    a.TMAX, 
    a.TMIN,  
    a.YEAR, 
    a.MONTH, 
    a.DAY, 
    b.over_max, 
    b.over_min 
    from autumn a join maxminall b
    on a.MONTH || a.DAY = b.concat WHERE YEAR = ''' + str(y)

#pd.read_sql(quer, conn)

In [17]:
conn = sqlite3.connect('weather.db')
c = conn.cursor()
c.execute('''CREATE TEMPORARY TABLE maxminall AS Select MONTH || DAY concat, MONTH, DAY,MAX(TMAX) over_max, MIN(TMIN) over_min 
            from weather GROUP BY 1, 2, 3 ORDER BY 2, 3''')
#Retrieves the range for the plots
rng = pd.read_sql('select MAX(TMAX) max, MIN(TMIN) min from weather', conn)
mx, mn =(rng.loc[0, 'max'], rng.loc[0,'min'])

pd.read_sql('Select * from weather where YEAR = 2018', conn).dropna()

,index,STATION,DATE,TMAX,TMIN,YEAR,MONTH,DAY
934,20751,USW00093821,2018-01-01 00:00:00,18.0,3.0,2018,1,1
935,20752,USW00093821,2018-01-02 00:00:00,22.0,0.0,2018,1,2
936,20753,USW00093821,2018-01-03 00:00:00,31.0,7.0,2018,1,3
937,20754,USW00093821,2018-01-04 00:00:00,24.0,7.0,2018,1,4
938,20755,USW00093821,2018-01-05 00:00:00,21.0,9.0,2018,1,5
939,20756,USW00093821,2018-01-06 00:00:00,23.0,4.0,2018,1,6
940,20757,USW00093821,2018-01-07 00:00:00,44.0,11.0,2018,1,7
941,20758,USW00093821,2018-01-08 00:00:00,41.0,35.0,2018,1,8
942,20759,USW00093821,2018-01-09 00:00:00,44.0,37.0,2018,1,9
943,20760,USW00093821,2018-01-10 00:00:00,63.0,42.0,2018,1,10
